# Обучение модели.
__Цель__: опредеделение стоимости игрока по его параметрам.<br/>
__Основная метрика__: MAE. Показывает среднее абсолютное отклонение между фактическими значениями целевой переменной и предсказаниями модели. Эта метрика хорошо интерпретируема и легко понятна, так как она показывает, на сколько единиц оценка модели отклоняется от реальных значений.<br/>
__Основные предположения__: Параметры, отражающие голевую и ассистенскую результативность должны сильно влиять на увеличении стоимости футболиста. Матчи и голы за наицональную сборную также должны иметь позитивную корреляцию со стоимостью. Красные и желтые карточки являются редким являением в футболе, их наличие или отсутствие не должно сильно влиять. Слишком возрастные футболисты стоят дешевле своих молодых коллег.

In [306]:
import numpy as np
import scipy.stats as sts

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor

import warnings
warnings.filterwarnings("ignore")

In [307]:
df = pd.read_csv("new_data.csv")

Определям параметры и целевую пермеенную.

In [308]:
target = "value"

params = ['age','height', 'national_matches',
       'national_goals', 'matches20', 'goals20', 'assists20', 'yellow20',
       'double_yellows20', 'red20', 'matches21', 'goals21', 'assists21',
       'yellow21', 'double_yellows21', 'red21', 'matches22', 'goals22',
       'assists22', 'yellow22', 'double_yellows22', 'red22', 'goalkeeper', 'deff', 'middle', 'attack',
          'goal_percentage20', 'goal_percentage21', 'goal_percentage22',
       'assists_percentage20', 'assists_percentage21', 'assists_percentage22']

Разделение выборки на тренировочную и тестовую.

In [309]:
df_train, df_test = train_test_split(df, test_size=0.2)

In [310]:
x_train, y_train =  df_train[params], df_train[target]
x_test, y_test = df_test[params], df_test[target]

# Наивная модель. 
Предположение что стоимость футболиста из тестовой выборки равна медианной для тренировочной:

In [311]:
y_pred_naive = y_train.median() * np.ones_like(y_test)

In [312]:
print(f"MSE: {mean_squared_error(y_test, y_pred_naive)}")
print(f"MAE: {mean_absolute_error(y_test, y_pred_naive)}")
print(f"MAPE: {mean_absolute_percentage_error(y_test, y_pred_naive)}")

MSE: 855429957.157258
MAE: 16728.125
MAPE: 2.059628503463795


Стандартизирование данных(приведение всех шкал к 1 размерности):

In [314]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)

# Линейная модель:

Подбираем лучший параметр для модели.

In [315]:
alphas = np.logspace(-2, 3, 20)
searcher = GridSearchCV(Ridge(), [{"alpha": alphas}], scoring="neg_mean_absolute_error", cv=10)
searcher.fit(x_train, y_train)
alpha = searcher.best_params_["alpha"]

In [316]:
alpha

162.3776739188721

In [317]:
model = Ridge(alpha=alpha)
model.fit(x_train, y_train)
y_pred_linear = model.predict(x_test)

In [318]:
print(f"MSE: {mean_squared_error(y_test, y_pred_linear)}")
print(f"MAE: {mean_absolute_error(y_test, y_pred_linear)}")
print(f"MAPE: {mean_absolute_percentage_error(y_test, y_pred_linear)}")

MSE: 331947831.8550523
MAE: 12263.430937434963
MAPE: 3.4409336486646245


Вес каждого признака:

In [319]:
pd.DataFrame({"признак":np.array(params),"вес":model.coef_})

,признак,вес
0,age,-6065.079146
1,height,975.983399
2,national_matches,6849.301308
3,national_goals,1471.296410
4,matches20,4769.368703
5,goals20,504.072238
6,assists20,1202.626188
7,yellow20,-526.807673
8,double_yellows20,198.432928
9,red20,-496.221402


# Выводы:
Линейная модель превосходит наивную по метрике MAE. Веса, которые посчитала модель,в основном реалистичны. Голы, ассисты матчи и матчи и голы за национальную сборную сильно позитивно влияют на стоимость. Но есть и необьяснимые показатели. Так, например, двойные желтые карточки за сезон 20 и красные карточки за 22 сезон увеличивают стоимость футболиста, а  процентаж голов за сезон 2020 уменьшает. Модель также считает, что игроки атаки стоят гораздо дешевле остальных, что противоречит изначальным предположениям, выдвинутым на основании графиков.Такой результат может гооворить о нелинейности связей между параметрами и целевой переменной. По модели можно заметить, что важность голевых показателей больше чем больше год, что логично, т.к. нынешнюю форму футболиста лучше отражают показатели последних лет. 

# Градиентный бустинг:

Подбираем лучшие параметры для модели.

In [320]:
params_boost = {
    'n_estimators': [50,100,150],
    'max_depth': [3,5,7],
    'learning_rate': [0.01,0.1,0.5],
}
model_boost = GradientBoostingRegressor()
grid_search = GridSearchCV(estimator=model_boost, param_grid=params_boost, cv=10, n_jobs=-1, scoring='neg_mean_absolute_error')
grid_search.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=GradientBoostingRegressor(), n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.1, 0.5],
                         'max_depth': [3, 5, 7],
                         'n_estimators': [50, 100, 150]},
             scoring='neg_mean_absolute_error')

In [321]:
grid_search.best_params_

{'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100}

In [322]:
rate = grid_search.best_params_["learning_rate"]
depth = grid_search.best_params_["max_depth"]
n = grid_search.best_params_["n_estimators"]

Обучение модели.

In [323]:
model_boost = GradientBoostingRegressor(n_estimators=n, max_depth=depth, learning_rate=rate, loss='huber')
model_boost.fit(x_train, y_train)
y_pred_boost = model_boost.predict(x_test)                                        

In [324]:
print(f"MSE: {mean_squared_error(y_test, y_pred_boost)}")
print(f"MAE: {mean_absolute_error(y_test, y_pred_boost)}")
print(f"MAPE: {mean_absolute_percentage_error(y_test, y_pred_boost)}")

MSE: 215903299.03978813
MAE: 8725.831399842866
MAPE: 1.4462995472980407


Вес каждого признака:

In [329]:
pd.DataFrame({"признак":np.array(params),"вес":model_boost.feature_importances_})

,признак,вес
0,age,0.067666
1,height,0.025790
2,national_matches,0.370496
3,national_goals,0.009944
4,matches20,0.078003
5,goals20,0.007612
6,assists20,0.006001
7,yellow20,0.012154
8,double_yellows20,0.001910
9,red20,0.001686


# Выводы:
Модель превосходит и наивную модель и линейную по метрике MAE. Результативные веса признаков выглядят более реалистично(принадлежность игрока линии атаки добавляет ему стоимости, карточки имеют относительно небольшой вес, т.к. это достаточно редкое явление на футбольном матче и т.п.). Самым влиятельным признаком, в соответствии с данной моделью, является количство матчей за национальную сборную. Большее влияние матчей, сыгранных в 21 году чем матчей, сыгранных в 22 можно  обьяснить тем, что рыночная стоимость еще не подстроилась под изменения в статистике футболиста на данный сезон и в большей степени опирается на прошлый. 

# Случайный лес:

Подбираем опттмальные параметры для модели.

In [330]:
params_for_forest = {
    "n_estimators": np.linspace(100, 1000, 20, dtype=int),
    "max_features": ['log2', 'sqrt'],
    "max_depth": np.linspace(1, 15, 15, dtype=int),
    "min_samples_split": np.linspace(2, 50, 10, dtype=int),
    "min_samples_leaf": np.linspace(2, 50, 10, dtype=int),
    "bootstrap": [True, False]
    }

In [331]:
searcher_forest = RandomizedSearchCV(RandomForestRegressor(),
                                     params_for_forest,
                                     n_iter=100,
                                     cv=4,
                                     scoring='roc_auc',
                                     n_jobs=-1,
                                     random_state=0,
                                     verbose=1)
searcher_forest.fit(x_train, y_train)

Fitting 4 folds for each of 100 candidates, totalling 400 fits


RandomizedSearchCV(cv=4, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15]),
                                        'max_features': ['log2', 'sqrt'],
                                        'min_samples_leaf': array([ 2,  7, 12, 18, 23, 28, 34, 39, 44, 50]),
                                        'min_samples_split': array([ 2,  7, 12, 18, 23, 28, 34, 39, 44, 50]),
                                        'n_estimators': array([ 100,  147,  194,  242,  289,  336,  384,  431,  478,  526,  573,
        621,  668,  715,  763,  810,  857,  905,  952, 1000])},
                   random_state=0, scoring='roc_auc', verbose=1)

In [332]:
searcher_forest.best_params_

{'n_estimators': 478,
 'min_samples_split': 18,
 'min_samples_leaf': 7,
 'max_features': 'log2',
 'max_depth': 3,
 'bootstrap': False}

In [333]:
n_estimators = searcher_forest.best_params_["n_estimators"]
max_features = searcher_forest.best_params_["max_features"]
max_depth = searcher_forest.best_params_["max_depth"]
min_samples_split = searcher_forest.best_params_["min_samples_split"]
min_samples_leaf = searcher_forest.best_params_["min_samples_leaf"]
bootstrap = searcher_forest.best_params_["bootstrap"]

Обучаем модель.

In [334]:
forest = RandomForestRegressor(n_estimators=n_estimators,
                             min_samples_split=min_samples_split,
                             min_samples_leaf=min_samples_leaf, max_features=max_features,
                             max_depth=max_depth,
                             bootstrap=bootstrap)
forest.fit(x_train, y_train)

RandomForestRegressor(bootstrap=False, max_depth=3, max_features='log2',
                      min_samples_leaf=7, min_samples_split=18,
                      n_estimators=478)

In [335]:
y_pred = forest.predict(x_test)

In [336]:
print(f"MSE: {mean_squared_error(y_test, y_pred)}")
print(f"MAE: {mean_absolute_error(y_test, y_pred)}")
print(f"MAPE: {mean_absolute_percentage_error(y_test, y_pred)}")

MSE: 359059552.7895552
MAE: 12993.070762355135
MAPE: 3.7707649012505295


Вес каждого признака:

In [337]:
pd.DataFrame({"признак":np.array(params),"вес":forest.feature_importances_})

,признак,вес
0,age,0.019839
1,height,0.001819
2,national_matches,0.218235
3,national_goals,0.101135
4,matches20,0.084048
5,goals20,0.019337
6,assists20,0.028693
7,yellow20,0.002298
8,double_yellows20,0.001734
9,red20,0.000057


# Выводы:
Модель случайного леса показала реузльтаты MAE хуже чем у линейной модели и модели бустинга. Но также как и в предыдущих моделях одни из главных признаков модель считает количество матчей за национальную сборную. Также как и модель бустинга модель случайного леса считает, что игроки линии атаки при прочих равных стоят дороже остальных.

# Общие выводы:
Лучше всего себя показала модель градиентного бустинга. Но даже ее результаты сложно назвать точными. Причины этого, возможно в недостаточном количестве данных.